In [67]:
"""    
    Indicador: Hectareaje X Hito.
"""

'    \n    Indicador: Hectareaje X Hito.\n'

<h3>Importación Librerías</h3>

In [68]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime as df

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_relacion_terrenos_w_ui
import funcion_hectareaje_general_ui

<h3>Parametrización de Variables</h3>

In [69]:
RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS"
DIRECTORIO_XLSX_2_2_4 = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_4_Hectareaje_X_Hito\zReportes\Seguimiento_Diario"

columnas_terrenos_w_ui = ['codigo','radicado','municipio','proceso','area_ha_cmt12','ID_UI','Zona_UI','Nombre_UI', 'Area_Ha_CMT12', 'Meta_Hito']
columnas_reducion_indicador_1 = ['Hito','Area_Ha_Contractual','hectareaje_visitado_bcgs']

fecha_actual = df.now()
fecha_directorio = str(fecha_actual.strftime("%Y-%m-%d"))

nombre_xlsx_2_2_4 = str(fecha_directorio) + '_indicador_hectareaje_avance_por_hito.xlsx'
RUTA_2_2_4 = os.path.join(DIRECTORIO_XLSX_2_2_4, nombre_xlsx_2_2_4)

ruta_fcTerrenos = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO"
ruta_bdScrach = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\Terrenos_w_UI.gdb"
ruta_fcUI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb\UI_Unificadas\UI_Unificadas"

<h3>Ejecución de Funciones</h3>

In [70]:

# ** Generación de ruta a la relación terreno ui
ruta_tlbTerrenoUI = funcion_relacion_terrenos_w_ui.funcion_terreno_w_ui(ruta_fcTerrenos, ruta_bdScrach, ruta_fcUI)

# ** Genera los DF de los estadísticos generales contractuales de hito vs Ha
df_hitos_ut, df_unidades_intervencion_hitos = funcion_hectareaje_general_ui.funcion_hectareaje_general_hito(ruta_fcUI)

1. Se genera centroides de Terrenos, capa fcTerrenosCentroides, base de datos Terrenos_w_UI.gdb
2. Se genera Join Terrenos Centroides W UI, capa capa_fccentroidesTerrenosWJoinUI, base de datos Terrenos_w_UI.gdb
3. Se borran campos en Terrenos Centroides W UI, capa capa_fccentroidesTerrenosWJoinUI, base de datos Terrenos_w_UI.gdb
4. Se genera Join Terrenos Centroides W Terrenos Originales, capa fcTerrenosUI, base de datos Terrenos_w_UI.gdb
5. Se borran campos en Terrenos W UI, capa fcTerrenosUI, base de datos Terrenos_w_UI.gdb
6.1. Se genera tabla relacion_terreno_ui que relaciona Terrenos con UI, base de datos Terrenos_w_UI.gdb
6.2. Se genera tabla relacion_terreno_ui que relaciona Terrenos con UI, base de datos BD_Consolidada_Lote4.gdb
FINALIZA PROCESO
Se crea dataset de capa UI_Unificadas
Se exporta C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\Estadisticos\indicadores_base_ui_lote_4.xlsx
FINALIZA PROCESO


<h3>Actualización</h3>

In [71]:

# ** Lectura del FC Generado en 2.2.0 {Terrenos Georreferenciados asociados a la Gestión BCGS}
df_terrenosAtencionBCGS = pd.DataFrame.spatial.from_featureclass(RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS)

# ? Se filtra Actualización
df_terrenosReconocimientoBCGS = df_terrenosAtencionBCGS[df_terrenosAtencionBCGS['proceso']=='Actualización']
# ? Copia de DF
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS.copy()
# ? Se extrae organizados propio de columnas
df_terrenosReconocimientoBCGS['id_actividad'] = df_terrenosReconocimientoBCGS['estado_tramite'].str[0:1].astype(int)
# ? Se filtra por indicador de proceso
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS[df_terrenosReconocimientoBCGS['id_actividad'] >= 6]

# ** Lectura del Relación Terreno - UI
df_relTerrenosUI = pd.DataFrame.spatial.from_table(ruta_tlbTerrenoUI)
# ** FC UI Completa
df_UI = pd.DataFrame.spatial.from_featureclass(ruta_fcUI)

# ** Unión, Terrenos con Tabla Relación Terrenos Vs UI
df_terrenosBCGS = pd.merge(left=df_terrenosReconocimientoBCGS,
    right=df_relTerrenosUI,
    left_on="codigo",
    right_on="codigo",
    how="inner"
    )

# ** Unión, Terrenos con ID_UI con FC de UI para completar datos de Hito
df_terrenosBCGS_w_UI = pd.merge(left=df_terrenosBCGS,
    right=df_UI,
    left_on = 'id_ui',
    right_on = 'ID_UI',
    how="inner"
    )

# ** Reducción de Columnas para mejorar análisis
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI[columnas_terrenos_w_ui]
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI_reducido.rename(columns={'Area_Ha_CMT12':'Area_Ha_CTM12_UI', 'area_ha_cmt12':'area_ha_ctm12_predio'})
print(f'{df_terrenosBCGS_w_UI_reducido.shape[0]}')
df_terrenosReconocimientoBCGS

307


,OBJECTID,codigo,codigo_anterior,area_ha_cmt12,radicado,npn,municipio,proceso,vr_nombre_proceso,estado_tramite,...,proceso_estado,vr_task_nom,vr_task_tip,vr_task_con,vr_task_est,codigo_zona,zona,municipio_estandarizado,SHAPE,id_actividad
71,72,086060002000000000243000000000,08606000200000243000,3.521193,3361,086060002000000000243000000000,REPELÓN,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,Repelon,"{""rings"": [[[4766889.2047, 2718759.4222999997]...",6
72,73,086060002000000000388000000000,08606000200000388000,3.075047,3362,086060002000000000388000000000,REPELÓN,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,Repelon,"{""rings"": [[[4766933.3594, 2718869.8353000004]...",6
73,74,086060002000000000387000000000,08606000200000387000,3.173331,3362,086060002000000000387000000000,REPELÓN,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,Repelon,"{""rings"": [[[4766986.2246, 2718994.1711], [476...",6
74,75,086060002000000000389000000000,08606000200000389000,3.091539,3362,086060002000000000389000000000,REPELÓN,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,Repelon,"{""rings"": [[[4766986.2246, 2718994.1711], [476...",6
75,76,086060002000000000267000000000,08606000200000267000,27.418469,3359,086060002000000000267000000000,REPELÓN,Actualización,Formación y Actualización,9.Realizando Transacción Catastral,...,Realizando Transacción Catastral,Transacción Catastral Alfanumérica,FNC,<NA>,<NA>,00,Rural,Repelon,"{""rings"": [[[4764014.4975, 2719399.4575999994]...",9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316,2317,134421600000000120006000000000,13442160000120006000,0.03382,2758,134421600000000120006000000000,MARÍA LA BAJA,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,16,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742791.1021, 2655283.7876999993]...",6
2317,2318,134421600000000120005000000000,13442160000120005000,0.012218,2758,134421600000000120005000000000,MARÍA LA BAJA,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,16,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742803.182, 2655319.8718], [4742...",6
2318,2319,134421600000000120007000000000,13442160000120007000,0.014695,2758,134421600000000120007000000000,MARÍA LA BAJA,Actualización,Formación y Actualización,6.Preparando Edición Geográfica,...,Preparando Edición Geográfica,Preparar edición geográfica,FNC,<NA>,<NA>,16,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742800.5604, 2655312.2097999994]...",6
2325,2326,134420900000000130001000000000,13442090000130001000,1.008665,2792,134420900000000130001000000000,MARÍA LA BAJA,Actualización,Formación y Actualización,7.Realizando Edición Geográfica,...,Realizando Edición Geográfica,Edición Geográfica,FNC,<NA>,<NA>,09,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742854.7317, 2657894.3812000006]...",7


<h3>Indicadores</h3>

In [72]:

# ** Indicador 1: Total Gestión BSCG (Activo) por Hito
df_terrenosBCGS_w_UI_indicador_1 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

# ** Indicador 2: Total Gestión BSCG (Activo) por Hito y UI
df_terrenosBCGS_w_UI_indicador_2 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito','ID_UI','Area_Ha_CTM12_UI'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

In [73]:
df_terrenosBCGS_w_UI_indicador_1

,Meta_Hito,hectareaje_visitado_bcgs
0,Hito 2,1160.81
1,Hito 3,906.604
2,Hito 4,77.229


In [74]:
df_indicador_1_contrato = pd.merge(left=df_hitos_ut,
    right=df_terrenosBCGS_w_UI_indicador_1,
    left_on = 'Hito',
    right_on = 'Meta_Hito',
    how="inner")

df_indicador_1_contrato_reducido = df_indicador_1_contrato[columnas_reducion_indicador_1]

df_indicador_1_contrato_reducido = df_indicador_1_contrato_reducido.copy()

df_indicador_1_contrato_reducido['porcentaje_visitado_bcgs'] = ((df_indicador_1_contrato_reducido['hectareaje_visitado_bcgs'] / df_indicador_1_contrato_reducido['Area_Ha_Contractual']) * 100).round(4)
df_indicador_1_contrato_reducido['hectareaje_predios_finalizados'] = 0

df_indicador_1_contrato_reducido

,Hito,Area_Ha_Contractual,hectareaje_visitado_bcgs,porcentaje_visitado_bcgs,hectareaje_predios_finalizados
0,Hito 2,16243.92,1160.81,7.1461,0
1,Hito 3,54146.40,906.604,1.6744,0
2,Hito 4,81219.60,77.229,0.0951,0


In [75]:
df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'] = df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'].round(3)
df_terrenosBCGS_w_UI_indicador_2['porcentaje_visitado_bcgs'] = ((df_terrenosBCGS_w_UI_indicador_2['hectareaje_visitado_bcgs'] / df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI']) * 100).round(4)
df_terrenosBCGS_w_UI_indicador_2['hectareaje_predios_finalizados'] = 0

df_terrenosBCGS_w_UI_indicador_2

,Meta_Hito,ID_UI,Area_Ha_CTM12_UI,hectareaje_visitado_bcgs,porcentaje_visitado_bcgs,hectareaje_predios_finalizados
0,Hito 2,ZI_MB_12,880.942,418.2,47.4719,0
1,Hito 2,ZI_MB_14,898.76,226.624,25.2152,0
2,Hito 2,ZI_MB_16,2153.949,295.079,13.6994,0
3,Hito 2,ZI_RP_05,2362.693,194.436,8.2294,0
4,Hito 2,ZI_RP_06,2180.411,26.471,1.214,0
5,Hito 3,ZI_MB_15_4,4.828,2.484,51.4499,0
6,Hito 3,ZI_MB_15_7,35.778,14.88,41.5898,0
7,Hito 3,ZI_MB_17_0,194.065,7.002,3.6081,0
8,Hito 3,ZI_RP_03,2189.123,882.238,40.301,0
9,Hito 4,ZI_RP_04,2894.664,11.125,0.3843,0


<h3>Generación XLSX</h3>

In [76]:
with pd.ExcelWriter(RUTA_2_2_4, engine='xlsxwriter') as writer:
    df_indicador_1_contrato_reducido.to_excel(writer, sheet_name='Indicador_1', index=False)
    df_terrenosBCGS_w_UI_indicador_2.to_excel(writer, sheet_name='Indicador_2', index=False)

print("Finaliza la exportación del XLSX")

Finaliza la exportación del XLSX
